In [43]:
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
import time

In [44]:
website = 'https://snappfood.ir/service/restaurant/city/semnan/near?page=0&superType=1'
path = 'chromedriver-win64/chromedriver.exe'
driver = webdriver.Chrome(service=Service(path))
driver.get(website)
time.sleep(5)
choose_address_button = driver.find_element('xpath', ('//button[@class="sc-fFubgz fQCnPg AutoAddressCard__Layout-sc-90emsr-0 egQPmP"]'))
choose_address_button.click()
time.sleep(2)
choose_semnan_button = driver.find_element('xpath', '//p[@class="sc-hKgILt esHHju"]')
choose_semnan_button.click()
time.sleep(2)
choose_semnan = driver.find_element('xpath', '//p[text()="سمنان"]')
choose_semnan.click()
time.sleep(2)
confirm = driver.find_element('xpath', '//button[text()= "تایید"]')
confirm.click()
time.sleep(5)

last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    # Scroll down to bottom
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)

    # Calculate new scroll height and compare with last scroll height
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

html = driver.page_source
soup = BeautifulSoup (html, "html.parser")
driver.close()
restaurant_links = []
box = soup.find('div', class_='sc-citwmv vwzHB')
href_list = box.find_all('a', class_='VendorCard__HtmlLink-sc-6qaz7-4 cdoeYu', href=True)
for link in href_list:
    restaurant_links.append(link['href'])

In [260]:
len(restaurant_links)

57

In [46]:
for link in restaurant_links:
    print(f'https://snappfood.ir{link}')

https://snappfood.ir/restaurant/menu/%D9%81%D8%B3%D8%AA_%D9%81%D9%88%D8%AF_%DA%A9%D9%84%D9%88%D8%A7-r-pvqxrm
https://snappfood.ir/restaurant/menu/%D9%81%D8%B3%D8%AA_%D9%81%D9%88%D8%AF_%D8%B3%D8%AA%D8%A7%D8%B1%D9%87_%D8%B4%D9%87%D8%B1-r-3x2rz9
https://snappfood.ir/restaurant/menu/%D9%81%D8%B3%D8%AA_%D9%81%D9%88%D8%AF_%D9%88%D8%B4%D9%86%D8%A7-r-3eqwzm
https://snappfood.ir/restaurant/menu/%D8%B3%D9%81%D8%B1%D9%87_%D8%AE%D8%A7%D9%86%D9%87_%D8%B3%D9%86%D8%AA%DB%8C_%D8%B6%DB%8C%D8%A7%D9%81%D8%AA_%D8%A7%D8%B1%DA%AF-r-32yyq2
https://snappfood.ir/restaurant/menu/%D8%B1%D8%B3%D8%AA%D9%88%D8%B1%D8%A7%D9%86_%D8%A7%DB%8C%D8%AA%D8%A7%D9%84%DB%8C%D8%A7%DB%8C%DB%8C_%D8%B1%DB%8C%D9%88%D8%A7%D9%86-r-37y849
https://snappfood.ir/restaurant/menu/%D8%B1%D8%B3%D8%AA%D9%88%D8%B1%D8%A7%D9%86_%D8%A7%D9%86%D8%AF%DB%8C%D8%B4%D9%87-r-0m8e4y
https://snappfood.ir/restaurant/menu/%D8%B1%D8%B3%D8%AA%D9%88%D8%B1%D8%A7%D9%86_%D8%A7%DB%8C%D9%88%D8%A7%D9%86-r-0gvw82
https://snappfood.ir/restaurant/menu/%D8%B1%D8%B3%D8%AA%

In [55]:
restaurant_info = pd.DataFrame(columns=['نام رستوران', 'دسته بندی', 'آدرس', 'شیوه پرداخت', 'حداقل سبد', 'مجموع امتیاز', 'میانگین امتیاز'])


In [139]:
# for i in range(len(restaurant_links)):
i = 56
root = 'https://snappfood.ir'
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--headless")
chrome_options.add_argument("user-agent=Chrome/119.0.6045.160")
driver = webdriver.Chrome(service=Service(path), options=chrome_options)
url = f'{root}{restaurant_links[i]}'
driver.get(url)
comments_button = driver.find_element('xpath', '//button[@class="ModalHeader__ActionButton-sc-ick1zw-1 hMDcUg"]')
comments_button.click()
time.sleep(1)
comments_button = driver.find_element('xpath', '//button[@class="sc-fFubgz hUpWbm"]')
comments_button.click()
time.sleep(1)
comments_button = driver.find_element('xpath', '//p[@class="sc-hKgILt jsaCNc"]')
comments_button.click()
time.sleep(1)
info_page_source = driver.page_source
driver.close()
soup = BeautifulSoup(info_page_source, 'lxml')
restaurant_info.loc[restaurant_links[i][-8:], 'نام رستوران'] = soup.find('p', class_= 'sc-hKgILt filZZV').get_text()
restaurant_info.loc[restaurant_links[i][-8:], 'دسته بندی'] = soup.find('p', class_= 'sc-hKgILt hmsjTi').get_text()
restaurant_info.loc[restaurant_links[i][-8:], 'آدرس'] = soup.find_all('p', class_= 'sc-hKgILt cnaQBH')[1].get_text()
restaurant_info.loc[restaurant_links[i][-8:], 'شیوه پرداخت'] = soup.find_all('p', class_= 'sc-hKgILt jsaCNc')[1].get_text()
# restaurant_info.loc[restaurant_links[i][-8:], 'حداقل سبد'] = soup.find_all('span', class_= 'sc-hKgILt hxREoh')[-1].get_text()
restaurant_info.loc[restaurant_links[i][-8:], 'مجموع امتیاز'] = soup.find('p', class_= 'sc-hKgILt fYlAbO').get_text()[1:-7]
restaurant_info.loc[restaurant_links[i][-8:], 'میانگین امتیاز'] = soup.find('span', class_= 'sc-hKgILt jsaCNc').get_text()

restaurant_info

,نام رستوران,دسته بندی,آدرس,شیوه پرداخت,حداقل سبد,مجموع امتیاز,میانگین امتیاز
r-pvqxrm,فست فود کلوا,"فست‌فود, پیتزا, ساندویچ, برگر",باغ فردوس، خیابان شهید صباغیان، فست فود کلوا,آنلاین,NaN,"۳,۸۱۹",۴.۲
r-3x2rz9,فست فود ستاره شهر,"فست‌فود, پیتزا, ساندویچ, برگر",بلوار ۱۷ شهریور، روبروی سرپرستی بانک ملی، فست ...,آنلاین,NaN,"۸,۶۵۵",۴.۴
r-3eqwzm,فست فود وشنا,"فست‌فود, پیتزا, ساندویچ, برگر",بلوار قدس، خیابان قدس، فست فود وشنا,آنلاین,NaN,"۷,۶۲۲",۴.۲
r-32yyq2,سفره خانه سنتی ضیافت ارگ,"ایرانی, کباب, گوشت, مرغ, برگر",میدان ارگ، خیابان شیخ فضل الله نوری، نبش خیابا...,آنلاین، اعتبار جایزه خرید,NaN,۶۰۸,۴.۴
r-37y849,رستوران ایتالیایی ریوان,"سالاد و پیش‌غذا, پیش غذا, پیتزا, پاستا و اسپاگتی",بلوار قائم، بازارچه قائم، رستوران ایتالیایی ریوان,آنلاین، اعتبار جایزه خرید,NaN,"۲,۶۵۰",۴.۳
r-0m8e4y,رستوران اندیشه,"ایرانی, کباب, چلو خورشت, پیش‌غذای ایرانی, مرغ",میدان سعدی، جنب شهرداری مرکزی، رستوران اندیشه,آنلاین,NaN,"۱,۶۵۸",۴.۴
r-0gvw82,رستوران ایوان,"ایرانی, کباب, پیش‌غذای ایرانی",مرکزی بلوار ولیعصر فرعی ۴ کوچه نصر ۳ طبقه همکف,آنلاین، اعتبار جایزه خرید,NaN,۳۳۷,۴.۳
r-pozvmk,رستوران دارچین,"صبحانه و میان‌وعده, صبحانه, سنتی, کباب, پیش‌غذ...",میدان مشاهیر، رستوران دارچین,آنلاین، اعتبار جایزه خرید,NaN,"۲,۱۷۶",۳.۸
r-o9q5dv,فست فود آیدا,"فست‌فود, ساندویچ",باغ فردوس، روبروی بانک گردشگری، ساختمان مهدی، ...,آنلاین، اعتبار جایزه خرید,NaN,۹,۴.۰
r-31qlyy,فست فود لونالانژ,"فست‌فود, پیتزا, صبحانه, برگر",بلوار قدس، باغ فامیلی (خیابان شورا)، بین فرعی ...,آنلاین، اعتبار جایزه خرید,NaN,"۴,۶۰۱",۴.۴


In [140]:
restaurant_info.to_csv('Restaurant-info.csv', encoding='utf-8-sig')

In [216]:
hours_of_work = pd.DataFrame(columns=['شنبه', 'یکشنبه', 'دوشنبه', 'سه‌شنبه', 'چهارشنبه', 'پنجشنبه', 'جمعه'])

In [274]:
for i in [26]:
    try:
        root = 'https://snappfood.ir'
        chrome_options = webdriver.ChromeOptions()
        chrome_options.add_argument("--headless")
        chrome_options.add_argument("user-agent=Chrome/119.0.6045.160")
        driver = webdriver.Chrome(service=Service(path), options=chrome_options)
        url = f'{root}{restaurant_links[i]}'
        driver.get(url)
        comments_button = driver.find_element('xpath', '//button[@class="ModalHeader__ActionButton-sc-ick1zw-1 hMDcUg"]')
        comments_button.click()
        time.sleep(1)
        comments_button = driver.find_element('xpath', '//button[@class="sc-fFubgz hUpWbm"]')
        comments_button.click()
        time.sleep(1)
        comments_button = driver.find_element('xpath', '//p[@class="sc-hKgILt jsaCNc"]')
        comments_button.click()
        time.sleep(1)
        info_page_source = driver.page_source
        driver.close()
        soup = BeautifulSoup(info_page_source, 'lxml')
        times = soup.find('div', class_= 'sc-cBNfnY VendorShifts__ShiftCard-sc-15229h3-0 jSOVBI HoDvy')
        daytimes = times.find_all('div', class_='sc-dlfnbm VendorShifts__DayContainer-sc-15229h3-1 kBWHgq cmIBz')
        for day in daytimes:
            value = []
            hour_ranges = day.find_all('div', 'sc-dlfnbm VendorShifts__TimeRange-sc-15229h3-2 gjUbvm bcjIOh')
            for hour in hour_ranges:
                start = hour.find_all('span', class_='sc-hKgILt sFIZX')[1].get_text()
                end = hour.find_all('span', class_='sc-hKgILt sFIZX')[0].get_text()
                value.append((start, end))
            
            hours_of_work.loc[restaurant_links[i][-8:], day.find('p', class_='sc-hKgILt kWRkVx').get_text()] = value

    except:
        print(i)


In [282]:
hours_of_work

,شنبه,یکشنبه,دوشنبه,سه‌شنبه,چهارشنبه,پنجشنبه,جمعه
r-pvqxrm,"[(۰۰:۰۰, ۰۰:۳۰), (۱۱:۰۰, ۲۳:۵۹)]","[(۰۰:۰۰, ۰۰:۳۰), (۱۱:۰۰, ۲۳:۵۹)]","[(۰۰:۰۰, ۰۰:۳۰), (۱۱:۰۰, ۲۳:۵۹)]","[(۰۰:۰۰, ۰۰:۳۰), (۱۱:۰۰, ۲۳:۵۹)]","[(۰۰:۰۰, ۰۰:۳۰), (۱۱:۰۰, ۲۳:۵۹)]","[(۰۰:۰۰, ۰۰:۳۰), (۱۱:۰۰, ۲۳:۵۹)]","[(۰۰:۰۰, ۰۰:۳۰), (۱۱:۰۰, ۲۳:۵۹)]"
r-3x2rz9,"[(۱۱:۰۰, ۲۳:۵۹)]","[(۱۱:۰۰, ۲۳:۵۹)]","[(۱۱:۰۰, ۲۳:۵۹)]","[(۱۱:۰۰, ۲۳:۵۹)]","[(۱۱:۰۰, ۲۳:۵۹)]","[(۱۱:۰۰, ۲۳:۵۹)]","[(۱۱:۰۰, ۲۳:۵۹)]"
r-3eqwzm,"[(۱۱:۰۰, ۲۳:۰۰)]","[(۱۱:۰۰, ۲۳:۰۰)]","[(۱۱:۰۰, ۲۳:۰۰)]","[(۱۱:۰۰, ۲۳:۰۰)]","[(۱۱:۰۰, ۲۳:۰۰)]","[(۱۱:۰۰, ۲۳:۰۰)]","[(۱۱:۰۰, ۲۳:۰۰)]"
r-32yyq2,"[(۱۲:۰۰, ۱۶:۰۰), (۱۹:۰۰, ۲۲:۳۰)]","[(۱۲:۰۰, ۱۶:۰۰), (۱۹:۰۰, ۲۲:۳۰)]","[(۱۲:۰۰, ۱۶:۰۰), (۱۹:۰۰, ۲۲:۳۰)]","[(۱۲:۰۰, ۱۶:۰۰), (۱۹:۰۰, ۲۲:۳۰)]","[(۱۲:۰۰, ۱۶:۰۰), (۱۹:۰۰, ۲۲:۳۰)]","[(۱۲:۰۰, ۱۶:۰۰), (۱۹:۰۰, ۲۲:۳۰)]","[(۱۲:۰۰, ۱۶:۰۰), (۱۹:۰۰, ۲۲:۳۰)]"
r-37y849,"[(۱۱:۳۰, ۲۳:۳۰)]","[(۱۱:۳۰, ۲۳:۳۰)]","[(۱۱:۳۰, ۲۳:۳۰)]","[(۱۱:۳۰, ۲۳:۳۰)]","[(۱۱:۳۰, ۲۳:۳۰)]","[(۱۱:۳۰, ۲۳:۳۰)]","[(۱۱:۳۰, ۲۳:۳۰)]"
r-0m8e4y,"[(۱۲:۰۰, ۱۵:۴۵), (۱۹:۰۰, ۲۲:۳۰)]","[(۱۲:۰۰, ۱۵:۴۵), (۱۹:۰۰, ۲۲:۳۰)]","[(۱۲:۰۰, ۱۵:۴۵), (۱۹:۰۰, ۲۲:۳۰)]","[(۱۲:۰۰, ۱۵:۴۵), (۱۹:۰۰, ۲۲:۳۰)]","[(۱۲:۰۰, ۱۵:۴۵), (۱۹:۰۰, ۲۲:۳۰)]","[(۱۲:۰۰, ۱۵:۴۵), (۱۹:۰۰, ۲۲:۳۰)]","[(۱۲:۰۰, ۱۵:۴۵), (۱۹:۰۰, ۲۲:۳۰)]"
r-0gvw82,"[(۱۰:۰۰, ۲۳:۰۰)]","[(۱۰:۰۰, ۲۳:۰۰)]","[(۱۰:۰۰, ۲۳:۰۰)]","[(۱۰:۰۰, ۲۳:۰۰)]","[(۱۰:۰۰, ۲۳:۰۰)]","[(۱۰:۰۰, ۲۳:۰۰)]","[(۱۰:۰۰, ۲۳:۰۰)]"
r-pozvmk,"[(۰۹:۰۰, ۲۲:۵۹)]","[(۰۹:۰۰, ۲۲:۵۹)]","[(۰۹:۰۰, ۲۲:۵۹)]","[(۰۹:۰۰, ۲۲:۵۹)]","[(۰۹:۰۰, ۲۲:۵۹)]","[(۰۹:۰۰, ۲۲:۵۹)]","[(۰۹:۰۰, ۲۲:۵۹)]"
r-o9q5dv,"[(۱۱:۰۰, ۲۳:۰۰)]","[(۱۱:۰۰, ۲۳:۰۰)]","[(۱۱:۰۰, ۲۳:۰۰)]","[(۱۱:۰۰, ۲۳:۰۰)]","[(۱۱:۰۰, ۲۳:۰۰)]","[(۱۱:۰۰, ۲۳:۰۰)]","[(۱۱:۰۰, ۲۳:۰۰)]"
r-31qlyy,"[(۰۸:۰۰, ۲۳:۵۹)]","[(۰۸:۰۰, ۲۳:۵۹)]","[(۰۸:۰۰, ۲۳:۵۹)]","[(۰۸:۰۰, ۲۳:۵۹)]","[(۰۸:۰۰, ۲۳:۵۹)]","[(۰۸:۰۰, ۲۳:۵۹)]","[(۰۸:۰۰, ۲۳:۵۹)]"


In [281]:
for row in hours_of_work.index:
    for col in hours_of_work.columns:
        if hours_of_work.loc[row, col] == []:
            hours_of_work.loc[row, col] = 'تعطیل'

In [279]:
f'{root}{restaurant_links[-9]}'

'https://snappfood.ir/restaurant/menu/%DA%A9%D8%A8%D8%A7%D8%A8%D8%B3%D8%B1%D8%A7%DB%8C_%D9%82%D8%A7%D8%A6%D9%85-r-0wq528'

In [284]:
hours_of_work.to_csv('Restaurant-hours-of-work.csv', encoding='utf-8-sig')

In [285]:
root = 'https://snappfood.ir'
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("user-agent=Chrome/119.0.6045.160")
driver = webdriver.Chrome(service=Service(path), options=chrome_options)
url = f'{root}{restaurant_links[0]}'
driver.get(url)
comments_button = driver.find_element('xpath', '//button[@class="ModalHeader__ActionButton-sc-ick1zw-1 hMDcUg"]')
comments_button.click()
time.sleep(1)
comments_button = driver.find_element('xpath', '//button[@class="sc-fFubgz hUpWbm"]')
comments_button.click()
time.sleep(1)
comments_button = driver.find_element('xpath', '//p[@class="sc-hKgILt jsaCNc"]')
comments_button.click()
time.sleep(1)